In [1]:
from pypozyx import *
import serial 
import numpy as np
from pypozyx.definitions.registers import *
from scipy import stats
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import random 
from openpyxl import Workbook
from openpyxl import load_workbook

write_wb = Workbook()
write_ws = write_wb.active

class GMM(object):
    def __init__(self, X, k=1):
        # dimension
        X = np.asarray(X)
        self.m, self.n = X.shape
        self.data = X.copy()
        # number of mixtures
        self.k = k
        
    def _init(self):
        # init mixture means/sigmas
        self.mean_arr = np.asmatrix(np.random.random((self.k, self.n)))
        self.sigma_arr = np.array([np.asmatrix(np.identity(self.n)) for i in range(self.k)])
        self.phi = np.ones(self.k)/self.k
        self.w = np.asmatrix(np.empty((self.m, self.k), dtype=float))
        #print(self.mean_arr)
        #print(self.sigma_arr)
    
    def fit(self, tol=1e-4):
        self._init()
        num_iters = 0
        ll = 1
        previous_ll = 0
        while(ll-previous_ll > tol):
            previous_ll = self.loglikelihood()
            self._fit()
            num_iters += 1
            ll = self.loglikelihood()
            #print('Iteration %d: log-likelihood is %.6f'%(num_iters, ll))
        #print('Terminate at %d-th iteration:log-likelihood is %.6f'%(num_iters, ll))
    
    def loglikelihood(self):
        ll = 0
        for i in range(self.m):
            tmp = 0
            for j in range(self.k):
                #print(self.sigma_arr[j])
                tmp += sp.stats.multivariate_normal.pdf(self.data[i, :], 
                                                        self.mean_arr[j, :].A1, 
                                                        self.sigma_arr[j, :]) *\
                       self.phi[j]
            ll += np.log(tmp) 
        return ll
    
    def _fit(self):
        self.e_step()
        self.m_step()
        
    def e_step(self):
        # calculate w_j^{(i)}
        for i in range(self.m):
            den = 0
            for j in range(self.k):
                num = sp.stats.multivariate_normal.pdf(self.data[i, :], 
                                                       self.mean_arr[j].A1, 
                                                       self.sigma_arr[j]) *\
                      self.phi[j]
                den += num
                self.w[i, j] = num
            self.w[i, :] /= den
            assert self.w[i, :].sum() - 1 < 1e-4
            
    def m_step(self):
        for j in range(self.k):
            const = self.w[:, j].sum()
            self.phi[j] = 1/self.m * const
            _mu_j = np.zeros(self.n)
            _sigma_j = np.zeros((self.n, self.n))
            for i in range(self.m):
                _mu_j += (self.data[i, :] * self.w[i, j])
                _sigma_j += self.w[i, j] * ((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
                #print((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
            self.mean_arr[j] = _mu_j / const
            self.sigma_arr[j] = _sigma_j / const
            #print(self.mean_arr)
            
def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat

import os
import psutil

def _check_usage_of_cpu_and_memory():
    
    pid = os.getpid()
    py  = psutil.Process(pid)
    
    cpu_usage   = os.popen("ps aux | grep " + str(pid) + " | grep -v grep | awk '{print $3}'").read()
    cpu_usage   = cpu_usage.replace("\n","")
    
    memory_usage  = round(py.memory_info()[0] /2.**30, 2)
    
    print("cpu usage\t\t:", cpu_usage, "%")
    print("memory usage\t\t:", memory_usage, "%")

bias = 0
std = 0.5
cdf = np.random.random(100)

for j in range(0,804):
    
    x = np.array([7.54,4.8])
    x = np.array([15*random.random(),15*random.random()])
    H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    d = x-H
    d = np.multiply(d,d)
    d = np.sqrt(d.sum(axis=1))

    d_col_1 = d[0]
    d_col_2 = d[1] 
    d_col_3 = d[2]
    d_col_4 = d[3]
    real_arr = np.array([d_col_1,d_col_2,d_col_3,d_col_4])

    zd = np.matrix([d_col_1,d_col_2,d_col_3,d_col_4])
    x_hat = tri(zd,H2)
    x_hat = x_hat.T+H[0]

    SAMPLE = 100
    
    load_wb = load_workbook("/Users/swbaek/Desktop/ranging_1.xlsx", data_only=True)
    #시트 이름으로 불러오기
    load_ws = load_wb['Sheet']
    zd_1=load_ws.cell(j+1,1).value
    zd_2=load_ws.cell(j+1,2).value
    zd_3=load_ws.cell(j+1,3).value
    zd_4=load_ws.cell(j+1,4).value
    
    zd_col_tmp_1 = zd_1#d[0] + np.random.normal(bias,std,1)
    zd_col_tmp_2 = zd_2#d[1] + np.random.normal(bias,std,1)
    zd_col_tmp_3 = zd_3#d[2] + np.random.normal(bias,std,1)
    zd_col_tmp_4 = zd_4#d[3] + np.random.normal(bias,std,1)

    X1 = zd_col_tmp_1 
    X2 = zd_col_tmp_2
    X3 = zd_col_tmp_3
    X4 = zd_col_tmp_4
    zd = np.matrix([zd_col_tmp_1,zd_col_tmp_2,zd_col_tmp_3,zd_col_tmp_4])

    X = np.array([zd_col_tmp_1,zd_col_tmp_2,zd_col_tmp_3,zd_col_tmp_4])

    zd_col_1 = zd_col_tmp_1 + np.random.normal(0,1,SAMPLE-1)
    zd_col_2 = zd_col_tmp_2 + np.random.normal(0,1,SAMPLE-1)
    zd_col_3 = zd_col_tmp_3 + np.random.normal(0,1,SAMPLE-1)
    zd_col_4 = zd_col_tmp_4 + np.random.normal(0,1,SAMPLE-1)
    mu_arr = np.array([zd_col_1,zd_col_2,zd_col_3,zd_col_4])
    
    X = np.append(X,mu_arr,axis = 1)
    X1 = np.append(X1,zd_col_1)
    X2 = np.append(X2,zd_col_2)
    X3 = np.append(X3,zd_col_3)
    X4 = np.append(X4,zd_col_4)
    X_rev = np.transpose(X)

    t = range(0,SAMPLE)
    gmm = GMM(X_rev)
    gmm.fit()
    dist_1 = gmm.mean_arr[0,0]
    dist_2 = gmm.mean_arr[0,1]
    dist_3 = gmm.mean_arr[0,2]
    dist_4 = gmm.mean_arr[0,3]
    zd = np.matrix([dist_1,dist_2,dist_3,dist_4])
    x_hat = tri(zd,H2)
    x_hat = x_hat.T+H[0]

    write_ws['A1'] = 'Result'
    write_ws.cell(j+1,1,x_hat[0][0])
    write_ws.cell(j+1,2,x_hat[0][1])

    write_wb.save('/Users/swbaek/Desktop/em_result.xlsx')#('/Users/swbaek/Desktop/deviation.xlsx')

AxisError: axis 1 is out of bounds for array of dimension 1

In [11]:
X_rev[range(0,SAMPLE)][0]

array([6.71983125, 6.54210089, 6.4610155 , 8.20932715])

In [105]:
print(gmm.mean_arr[0,0])

4.763750551053947


In [11]:
print(X)

[0.    5.009 5.047 6.335]


In [1]:
print(x_hat[1])

NameError: name 'x_hat' is not defined

In [3]:
import numpy as np
x = np.array([7.54,4.8])

H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
d = x-H
d = np.multiply(d,d)
d = np.sqrt(d.sum(axis=1))

d_col_1 = d[0]
d_col_2 = d[1] 
d_col_3 = d[2]
d_col_4 = d[3]
real_arr = np.array([d_col_1,d_col_2,d_col_3,d_col_4])

zd = np.matrix([d_col_1,d_col_2,d_col_3,d_col_4])
print(zd)

[[4.8        2.41       8.16088231 7.161592  ]]
